# Ejemplo descriptores globales para imágenes color

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 25 de marzo de 2025

En este ejemplo se muestra algunas formas simples para obtener un descriptor de una imagen color.

Este ejemplo es muy similar al de descriptores para imágenes en gris: Las imágenes en la carpeta imágenes se comparan todas entre sí y para cada una se muestra la más parecida.

La comparación es por medio de histogramas de color:

   1. **Vector de colores**: solo reducir la imagen RGB
   2. **Histogramas por zona y por canal (3 x 1D)**: Representa distribución de cada canal r, g, b como histogramas de gris. No funciona muy bien porque no representa colores
   3. **Histogramas por zona de color (3D)**: Representa cada zona por un histograma de color y compara histogramas con distancia L1
   4. **Histogramas 3D + distancia EMD**: El mismo anterior pero comparando con distancia EMD  

Cada ejemplo tiene un parámetro `mostrar_imagenes=True` que se puede poner en `False` para probar más rápido.

En cada experimento se muestra un porcentaje de respuestas correctas (imágenes en que el descriptor más cercano es igual a la imagen más parecida).

### Primero unas funciones auxiliares

In [ ]:
import numpy
import cv2
import os
import time
import scipy.spatial
import pandas

def listar_archivos_en_carpeta(imagenes_dir):
    lista = []
    for archivo in os.listdir(imagenes_dir):
        # los que terminan en .jpg se agregan a la lista de nombres
        if archivo.endswith(".jpg"):
            lista.append(os.path.join(imagenes_dir, archivo))
    lista.sort()
    return lista

def agregar_texto(imagen, texto):
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.7
    fontThickness = 2
    position = (10, 30)
    fontColor = (50, 50, 50)
    cv2.putText(imagen, texto, position, fontFace, fontScale, fontColor, fontThickness, cv2.LINE_AA)

def mostrar_imagen(window_name, imagen, filename):
    MIN_WIDTH, MAX_WIDTH = 100, 1920
    MIN_HEIGHT, MAX_HEIGHT = 100, 1080
    if imagen.shape[0] > MAX_HEIGHT or imagen.shape[1] > MAX_WIDTH:
        # reducir tamaño
        fh = MAX_HEIGHT / imagen.shape[0]
        fw = MAX_WIDTH / imagen.shape[1]
        escala = min(fh, fw)
        imagen = cv2.resize(imagen, (0, 0), fx=escala, fy=escala, interpolation=cv2.INTER_CUBIC)
    elif imagen.shape[0] < MIN_HEIGHT or imagen.shape[1] < MIN_WIDTH:
        # aumentar tamaño
        fh = MIN_HEIGHT / imagen.shape[0]
        fw = MIN_WIDTH / imagen.shape[1]
        escala = max(fh, fw)
        imagen = cv2.resize(imagen, (0, 0), fx=escala, fy=escala, interpolation=cv2.INTER_NEAREST)
    # incluir el nombre sobre la imagen
    agregar_texto(imagen, os.path.basename(filename))
    # mostrar en pantalla
    cv2.imshow(window_name, imagen)

#funcion base para calcular un descriptor para cada imagen
def calcular_descriptores(funcion_descriptor, archivos_imagenes, mostrar_resultado):
    matriz_descriptores = None
    num_fila = 0
    for archivo_imagen in archivos_imagenes:
        # con cv2.IMREAD_COLOR la imagen leida siempre tiene 3 canales BGR
        imagen = cv2.imread(archivo_imagen, cv2.IMREAD_COLOR)
        if imagen is None:
            raise Exception("no puedo abrir: " + archivo_imagen)
        # la imagen donde se visusaliza el descriptor
        imagen_mostrar = None
        if mostrar_resultado:
            imagen_mostrar = numpy.zeros((imagen.shape[0], imagen.shape[1], 3), dtype=numpy.uint8)
        # llama la funcion que calcula el descriptor
        descriptor_imagen = funcion_descriptor(imagen, imagen_mostrar)
        # mostrar la visualizacion
        if imagen_mostrar is not None:
            mostrar_imagen("imagen", imagen, archivo_imagen)
            mostrar_imagen("descriptor", imagen_mostrar, archivo_imagen)
            key = cv2.waitKey()
            cv2.destroyAllWindows()
            # salir con 'q' o con ESC
            if key == ord('q') or key == 27:
                raise KeyboardInterrupt            
        # crear la matriz de descriptores (numero imagenes x largo_descriptor)
        if matriz_descriptores is None:
            matriz_descriptores = numpy.zeros((len(archivos_imagenes), len(descriptor_imagen)), numpy.float32)
        elif len(descriptor_imagen) != matriz_descriptores.shape[1]:
            raise Exception("descriptor de largo {} != {}".format(len(descriptor_imagen), matriz_descriptores.shape[1]))
        # copiar descriptor a una fila de la matriz de descriptores
        matriz_descriptores[num_fila] = descriptor_imagen
        num_fila += 1
    return matriz_descriptores

# funcion que divide un rango [0, max) en  intervalos.
# Por ej: limites(256, 2) -> [0, 128, 256] que define los rangos [0, 128) y [128, 256)
# Se usa para definir los rangos de cada bin de histograma o para dividir la imagen en zonas
def calcular_limites(maximo_no_incluido, cantidad):
    list = [round(maximo_no_incluido * i / cantidad) for i in range(cantidad)]
    list.append(maximo_no_incluido)
    return list


#funcion base para descriptores globales que se calculan por zonas rectangulares
def descriptor_por_zona_generico(imagen, num_zonas_x, num_zonas_y, funcion_descriptor_zona, imagen_mostrar):
    if imagen_mostrar is not None:
        cuadriculado(imagen_mostrar)
    descriptor = []
    limites_y = calcular_limites(imagen.shape[0], num_zonas_y)
    limites_x = calcular_limites(imagen.shape[1], num_zonas_x)
    for j in range(num_zonas_y):
        desde_y = limites_y[j]
        hasta_y = limites_y[j + 1]
        for i in range(num_zonas_x):
            desde_x = limites_x[i]
            hasta_x = limites_x[i + 1]
            # recortar la zona de la imagen a la que se calcula el descriptor
            zona = imagen[desde_y:hasta_y, desde_x:hasta_x]
            # recortar la zona imagen de visualizacion para mostrar el resultado del descriptor
            zona_mostrar = None
            if imagen_mostrar is not None:
                cv2.rectangle(imagen_mostrar, (desde_x,desde_y), (hasta_x-1,hasta_y-1), (0,0,0), 3)
                zona_mostrar = imagen_mostrar[desde_y:hasta_y, desde_x:hasta_x]
            # descriptor de la zona
            descriptor_zona = funcion_descriptor_zona(zona, zona_mostrar)
            # agregar descriptor de la zona al descriptor global
            descriptor.extend(descriptor_zona)
    # como visualizacion marco las zonas en la imagen original para entender qué zona representa cada descriptor
    if imagen_mostrar is not None:
        for j in range(num_zonas_y):
            desde_y = limites_y[j]
            hasta_y = limites_y[j + 1]
            for i in range(num_zonas_x):
                desde_x = limites_x[i]
                hasta_x = limites_x[i + 1] 
                cv2.rectangle(imagen, (desde_x,desde_y), (hasta_x - 1, hasta_y - 1), (0,0,0), 2)
    return descriptor

# para crear una imagen base
def cuadriculado(imagen, size_cuadros=6, cuadro_color1=(240, 200, 200), cuadro_color2=(190, 240, 190)):
    flag1 = True
    for y in range(0, imagen.shape[0], size_cuadros):
        flag2 = not flag1
        for x in range(0, imagen.shape[1], size_cuadros):
            tl = (x, y)
            br = (x + size_cuadros, y + size_cuadros)
            col = cuadro_color1 if flag2 else cuadro_color2
            cv2.rectangle(imagen, tl, br, col, -1)
            flag2 = not flag2
        flag1 = not flag1

# cada barra del histograma debe tener definido su color
# para mejor visualización se está escalando el valor máximo a la altura máxima
def dibujar_histograma(img, histograma, color_bins):
    pos_y_base = img.shape[0] - 4
    max_altura = img.shape[0] - 12
    nbins = len(histograma)
    max_val = numpy.max(histograma)
    limites_x = calcular_limites(img.shape[1], nbins)
    for i in range(nbins):
        desde_x = limites_x[i]
        hasta_x = limites_x[i + 1] - 1
        altura = round(histograma[i] / max_val * max_altura)
        pt1 = (desde_x, pos_y_base + 2)
        pt2 = (hasta_x, pos_y_base - altura)
        cv2.rectangle(img, pt1, pt2, color_bins[i], -1)

# compara una matriz de descriptores (vectores como fila) con una funcion de distancia
# usa cdist() con distancia L1
def comparar_todos_distancia_L1(matriz_descriptores):
    matriz_distancias = scipy.spatial.distance.cdist(matriz_descriptores, matriz_descriptores, metric='cityblock')
    return matriz_distancias

# imprime para imagen de consulta en una  fila cual es su más cercano
def imprimir_mas_cercanos(matriz_distancias, archivos_imagenes):
    # completar la diagonal con un valor muy grande para que el mas cercano no sea si mismo
    numpy.fill_diagonal(matriz_distancias, numpy.inf)
    # obtener la posicion del mas cercano por fila
    posiciones_minimas = numpy.argmin(matriz_distancias, axis=1)
    valores_minimos = numpy.amin(matriz_distancias, axis=1)
    tabla_resultados = []
    total = 0
    buenas = 0
    for i in range(len(matriz_distancias)):
        query = os.path.basename(archivos_imagenes[i])
        distancia = valores_minimos[i]
        mas_cercano = os.path.basename(archivos_imagenes[posiciones_minimas[i]])
        ok = query[:-5] == mas_cercano[:-5] #sacar los ultimos n.jpg
        tabla_resultados.append([query, mas_cercano, distancia, ok])
        total += 1
        if ok:
            buenas += 1
    df = pandas.DataFrame(tabla_resultados, columns=["query", "más_cercana", "distancia", "ok?"])
    print(df.to_string(index=False, justify='center'))
    print()
    print("buenas: {} / {} = {:.1f}%".format(buenas, total, 100*buenas/total))

# proceso completo: 1-leer imagenes, 2-calcular descriptores, 3-comparar descriptores, 4-mostrar los mas cercanos
def comparar_imagenes(funcion_descriptor, funcion_comparar_todos, imagenes_dir, mostrar_imagenes):
    archivos_imagenes = listar_archivos_en_carpeta(imagenes_dir)
    print("imágenes ({})={}".format(len(archivos_imagenes), [os.path.basename(filename) for filename in archivos_imagenes]))
    print()
    t0 = time.time()
    matriz_descriptores = calcular_descriptores(funcion_descriptor, archivos_imagenes, mostrar_imagenes)
    t1 = time.time()
    print("matriz descriptores={}, tiempo descriptores= {:.1f} segundos".format(matriz_descriptores.shape, t1 - t0))
    matriz_distancias = funcion_comparar_todos(matriz_descriptores)
    t2 = time.time()
    print("matriz distancias={}, tiempo distancias= {:.1f} segundos".format(matriz_distancias.shape, t2 - t1))
    print()
    imprimir_mas_cercanos(matriz_distancias, archivos_imagenes)


## 1. Vector de colores


In [ ]:
def vector_de_colores(imagen, imagen_mostrar):
    global size_x, size_y
    imagen_reducida = cv2.resize(imagen, (size_x, size_y), interpolation=cv2.INTER_AREA)
    # flatten convierte la matriz de w x h x 3 en un array de largo w x h x 3
    descriptor_imagen = imagen_reducida.flatten()
    # mostrar el descriptor
    if imagen_mostrar is not None:
        cv2.resize(imagen_reducida, (imagen.shape[1],imagen.shape[0]), dst=imagen_mostrar, interpolation=cv2.INTER_NEAREST)
    return descriptor_imagen

size_x = 3
size_y = 3

comparar_imagenes(vector_de_colores, comparar_todos_distancia_L1, "imagenes", mostrar_imagenes=True)


## 2. Histogramas por zona y por canal (3 x 1D)

Corresponde a concatenar tres histogramas de gris, cada uno con la distribuciones de valores para los canales R, G y B.

Notar que no hay representación de colores porque cada histograma representa cada canal en forma independiente.

In [ ]:
def color_de_cada_bin_3x1d(cantidad_bins):
    limites_gris = calcular_limites(256, cantidad_bins)
    grises = [round((limites_gris[i] + limites_gris[i + 1] - 1) / 2) for i in range(cantidad_bins)]
    colores_r = [(0, 0, gris) for gris in grises]
    colores_g = [(0, gris, 0) for gris in grises]
    colores_b = [(gris, 0, 0) for gris in grises]
    return (colores_r, colores_g, colores_b)
   
def calcular_histograma_1d(imagen_gris, cantidad_bins):
    hist = cv2.calcHist(images=[imagen_gris], channels=[0], mask=None, histSize=[cantidad_bins], ranges=(0, 256))
    array = hist.flatten()
    return array / numpy.sum(array)

def histograma_por_canal_3x1d(imagen_zona, imagen_zona_mostrar):
    global cantidad_bins_3x1d
    # separa la imagen en imagenes independiente por canal
    canales_bgr = cv2.split(imagen_zona)
    # un histograma (array de numeros) para cada canal
    histograma_b = calcular_histograma_1d(canales_bgr[0], cantidad_bins_3x1d)
    histograma_g = calcular_histograma_1d(canales_bgr[1], cantidad_bins_3x1d)
    histograma_r = calcular_histograma_1d(canales_bgr[2], cantidad_bins_3x1d)
    # el descriptor es unir los tres arrays
    descriptor_zona = []
    descriptor_zona.extend(histograma_r)
    descriptor_zona.extend(histograma_g)
    descriptor_zona.extend(histograma_b)
    if imagen_zona_mostrar is not None:
        # division vertical
        limites_h = calcular_limites(imagen_zona_mostrar.shape[0], 3)
        zona_r = imagen_zona_mostrar[limites_h[0]:limites_h[1], : ]
        zona_g = imagen_zona_mostrar[limites_h[1]:limites_h[2], : ]
        zona_b = imagen_zona_mostrar[limites_h[2]:limites_h[3], : ]
        # colores
        (colores_r, colores_g, colores_b) = color_de_cada_bin_3x1d(cantidad_bins_3x1d)
        # dibujar los tres histogramas
        dibujar_histograma(zona_r, histograma_r, colores_r)
        dibujar_histograma(zona_g, histograma_g, colores_g)
        dibujar_histograma(zona_b, histograma_b, colores_b)
    return descriptor_zona

def histograma_por_canal_por_zonas(imagen, imagen_mostrar):
    global num_zonas_x_3x1d, num_zonas_y_3x1d
    descriptor_imagen = descriptor_por_zona_generico(imagen, zonas_x_3x1d, zonas_y_3x1d, histograma_por_canal_3x1d, imagen_mostrar)
    return descriptor_imagen

zonas_x_3x1d = 3
zonas_y_3x1d = 3
cantidad_bins_3x1d = 64

comparar_imagenes(histograma_por_canal_por_zonas, comparar_todos_distancia_L1, "imagenes", mostrar_imagenes=True)


## 3. Histogramas por zona de color (3D)

In [ ]:
def color_de_cada_bin_3d(cantidad_bins):
    limites_dim = calcular_limites(256, cantidad_bins)
    colores_bins = []
    for i in range(cantidad_bins):
        val1 = round((limites_dim[i] + limites_dim[i + 1] - 1) / 2)
        for j in range(cantidad_bins):
            val2 = round((limites_dim[j] + limites_dim[j + 1] - 1) / 2)
            for k in range(cantidad_bins):
                val3 = round((limites_dim[k] + limites_dim[k + 1] - 1) / 2)
                colores_bins.append((val1, val2, val3))
    return colores_bins
                    
def histograma_3d(imagen_zona, imagen_zona_mostrar):
    global cantidad_bins_3d
    hist = cv2.calcHist(images=[imagen_zona], channels=[0, 1, 2], mask=None, histSize=[cantidad_bins_3d,cantidad_bins_3d,cantidad_bins_3d], ranges=[0, 256, 0, 256, 0, 256])
    descriptor_zona = hist.flatten()
    descriptor_zona = descriptor_zona / numpy.sum(descriptor_zona)
    if imagen_zona_mostrar is not None:       
        colores_bins = color_de_cada_bin_3d(cantidad_bins_3d)
        dibujar_histograma(imagen_zona_mostrar, descriptor_zona, colores_bins)
    return descriptor_zona


def histograma_3d_por_zonas(imagen, imagen_mostrar):
    global zonas_x_3d, zonas_y_3d
    descriptor_imagen = descriptor_por_zona_generico(imagen, zonas_x_3d, zonas_y_3d, histograma_3d, imagen_mostrar)
    return descriptor_imagen


zonas_x_3d = 3
zonas_y_3d = 3
cantidad_bins_3d = 3

comparar_imagenes(histograma_3d_por_zonas, comparar_todos_distancia_L1, "imagenes", mostrar_imagenes=True)


## 4. Histogramas 3D + distancia EMD

In [ ]:
cost_matrix = None

def get_cost_matrix():
    global cost_matrix, cantidad_bins_3d
    if cost_matrix is None:
        colores_bins = color_de_cada_bin_3d(cantidad_bins_3d)
        cost_matrix = numpy.zeros((len(colores_bins), len(colores_bins)), dtype=numpy.float32)
        for i in range(len(colores_bins)):
            color1 = numpy.array(colores_bins[i])
            for j in range(len(colores_bins)):
                color2 = numpy.array(colores_bins[j])
                # distancia euclidiana entre vectoresn en el espacio RGB
                # se podria usar distancia en CIELAB
                cost_matrix[i, j] = numpy.linalg.norm(color1 - color2)
    return cost_matrix
        
def calcular_emd(hist_1, hist_2):
    # calcular la matriz de costos entre colores
    cost_matrix = get_cost_matrix()
    assert cost_matrix.shape[0] == len(hist_1)
    assert cost_matrix.shape[1] == len(hist_2)
    # La documentacion de EMD está en https://docs.opencv.org/4.10.0/d6/dc7/group__imgproc__hist.html#ga902b8e60cc7075c8947345489221e0e0
    # la versión python se llama wrapperEMD()
    # EMD con distType=cv2.DIST_USER usa la matriz de costos como parámetro (hist1_ e hist_2 son arrays)
    # retorna la distancia y los flujos
    # EMD con cv.DIST_L2 calcula la matriz de costos usando con distancia euclidiana entre colores (similar a este ejemplo)
    # en este caso requiere que cada bin del histograma contenga el peso y el color que representa, i.e. matriz de nx4, n filas, cada fila con [peso,r,g,b]
    distance, _, flow_matrix = cv2.EMD(hist_1, hist_2, distType=cv2.DIST_USER, cost=cost_matrix)
    return distance

def distancia_emd_descriptores(descriptor_1, descriptor_2):
    global zonas_x_3d, zonas_y_3d
    num_zonas = zonas_x_3d * zonas_y_3d
    # ambos descriptores del mismo largo, aunque EMD puede comparar distinto largo
    assert len(descriptor_1) == len(descriptor_2)
    length_zona = int(len(descriptor_1) / num_zonas)
    # como es un descriptor por zona, se calcula la distancia entre cada descriptor y se suman
    sum = 0
    for desde in range(0, len(descriptor_1), length_zona):
        parte1 = descriptor_1[desde : desde + length_zona]
        parte2 = descriptor_2[desde : desde + length_zona]
        sum += calcular_emd(parte1, parte2)
    return sum  

def comparar_todos_distancia_EMD(matriz_descriptores):
    num_descriptores = matriz_descriptores.shape[0]
    matriz_distancias = numpy.zeros((num_descriptores, num_descriptores), dtype=numpy.float32)
    for i in range(num_descriptores):
        for j in range(i + 1, num_descriptores):
            val = distancia_emd_descriptores(matriz_descriptores[i], matriz_descriptores[j])
            # la distancia es simetrica
            matriz_distancias[j, i] = matriz_distancias[i, j] = val
    return matriz_distancias

# histograma global de 9x9x9=729 colores
zonas_x_3d = 1
zonas_y_3d = 1
cantidad_bins_3d = 9

comparar_imagenes(histograma_3d_por_zonas, comparar_todos_distancia_EMD, "imagenes", mostrar_imagenes=True)



**Pregunta:** En resumen, ¿qué descriptor permite asociar mejor una imagen con su más parecida?